### Circle における学生の乗車 Group 分け問題

#### 課題整理
「学生をどの車に割り当てるかを決定する」ので, **「0-1 整数計画問題」** となる

- 学生の乗車 Group 分け問題
    1. 学生をどの車に割り当てるかを決定する
- 旅行に行くための法規制に関する制約
    2. 乗車人数は定員を超えてはいけない
    3. 運転免許証を持つ学生を１人以上、車に割り当てる
- 懇親を目的とした制約
    4. 各学年の学生を１人以上、車に割り当てる
- Gender Balance を考慮した制約
    5. 男女それぞれ１人以上を車に割り当てる


###### Topick
**0-1 整数計画問題**
変数が 0 か 1 をとる問題

In [1]:
import pandas as pd  # Data を読み込み、操作するための Library

students_df = pd.read_csv('resource/students.csv')
print(students_df.shape)  # Data の行数、列数を確認

(24, 4)


In [2]:
# 上位 5 件を表示
students_df.head()

,student_id,license,gender,grade
0,0,0,0,1
1,1,1,1,2
2,2,1,0,3
3,3,1,1,4
4,4,0,1,1


Column | 名称 | 説明
--- | --- | ---
student_id | 学生 ID | 0 ~ 23 の間で Unique な整数値をとる
license | 運転免許証有無 | 0: 未所持, 1: 所持
gender | 性別 | 0: 男性, 1: 女性
grade | 学年 | 1 ~ 4 の整数値をとる

In [3]:
cars_df = pd.read_csv('resource/cars.csv')
print(cars_df.shape)  # Data の行数, 列数を確認

(6, 2)


In [4]:
cars_df

,car_id,capacity
0,0,6
1,1,6
2,2,5
3,3,4
4,4,5
5,5,5


Column | 名称 | 説明
--- | --- | ---
car_id | 車 ID | 0~5 の間で Unique な整数値をとる
capacity | 乗車定員 | 4, 5, 6 の整数値をとる

#### 数理 Modeling
![数理 Modeling](mathematical_modeling.png)

- 入力: students.csv と cars.csv から得られる List と定数
- 出力: 各学生 *s* が どの車 *c* に乗ればよいかを表す変数 *x_sc*

### 数理 Model の実装

In [5]:
import pandas as pd
import pulp

students_df = pd.read_csv('resource/students.csv')
cars_df = pd.read_csv('resource/cars.csv')

# 学生の乗車 Group 分け問題（0-1 整数計画問題）の Instance 作成
prob = pulp.LpProblem('ClubCarProblem', pulp.LpMinimize)

In [6]:
# List
S = students_df['student_id'].to_list()  # 学生の List
C = cars_df['car_id'].to_list()  # 車の List
G = [1, 2, 3, 4]  # 学年の List
SC = [(s, c) for s in S for c in C]  # 学生の車の Pair の List
S_license = students_df[students_df['license'] == 1]['student_id']  # 免許を持っている学生の List
S_g = {g: students_df[students_df['grade'] == g]['student_id'] for g in G}  # 学年が g の学生の List
S_male = students_df[students_df['gender'] == 0]['student_id']
S_female = students_df[students_df['gender'] == 1]['student_id']

In [7]:
# 定数
# 車の乗車定員の定数
U = cars_df['capacity'].to_list()

In [8]:
# 変数
# 学生をどの車に割り当てるかを変数として定義
x = pulp.LpVariable.dicts('x', SC, cat='Binary')

In [9]:
# 制約
# 1. 各学生を１つの車に割り当てる
for s in S:
    prob += pulp.lpSum([x[s, c] for c in C]) == 1

# 2. 法規制に関する制約: 各車には乗車定員より多く乗ることができない
for c in C:
    prob += pulp.lpSum([x[s, c] for s in S]) <= U[c]

# 3. 法規制に関する制約: 各車に Driver を１人以上割り当てる
for c in C:
    prob += pulp.lpSum([x[s, c] for s in S_license]) >= 1

# 4. 懇親を目的とした制約: 各車に各学年の学生を１人以上割り当てる
for c in C:
    for g in G:
        prob += pulp.lpSum([x[s, c] for s in S_g[g]]) >= 1

# 5. Gender balance を考慮した制約: 各車に男性を１人以上割り当てる
for c in C:
    prob += pulp.lpSum([x[s, c] for s in S_male]) >= 1

# 6. Gender balance を考慮した制約: 各車に女性を１人以上割り当てる
for c in C:
    prob += pulp.lpSum([x[s, c] for s in S_female]) >= 1

### 問題を解く

In [10]:
# 求解
status = prob.solve()
print('Status:', pulp.LpStatus[status])

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/tera/.local/share/virtualenvs/PyMathematicalOptimization-moAdKnc7/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/linux/64/cbc /tmp/2d863d9dafb94821ab041fed3c495330-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/2d863d9dafb94821ab041fed3c495330-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 77 COLUMNS
At line 1039 RHS
At line 1112 BOUNDS
At line 1258 ENDATA
Problem MODEL has 72 rows, 145 columns and 672 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.00 seconds
Cgl0005I 24 SOS with 144 members
Cgl0004I processed model has 72 rows, 144 columns (144 integer (144 of which binary)) and 672 elements
Cbc0038I Initial state - 29 integers unsatisfied sum - 8.8
Cbc0038I Pass   1: suminf.    0.00000 (0) obj. 0 iterations 30
Cbc0038I Solution found of 0
Cbc0038

In [11]:
# 最適化結果の表示
# 各車に割り当てられている学生の List を辞書に格納（車 ID -> 割り当てられた学生の List）
car2students = {c: [s for s in S if x[s, c].value() == 1] for c in C}

# 各車の乗車定員（車 ID -> 乗車定員）
max_people = dict(zip(cars_df['car_id'], cars_df['capacity']))

for c, ss in car2students.items():
    print(f"車 ID: {c}")
    print(f"学生数(乗車定員): {len(ss)}({max_people[c]})")
    print(f"学生: {ss}\n")

車 ID: 0
学生数(乗車定員): 4(6)
学生: [6, 7, 8, 9]

車 ID: 1
学生数(乗車定員): 4(6)
学生: [4, 18, 21, 23]

車 ID: 2
学生数(乗車定員): 4(5)
学生: [5, 19, 20, 22]

車 ID: 3
学生数(乗車定員): 4(4)
学生: [3, 10, 13, 16]

車 ID: 4
学生数(乗車定員): 4(5)
学生: [0, 2, 11, 17]

車 ID: 5
学生数(乗車定員): 4(5)
学生: [1, 12, 14, 15]



#### Python から API に request を投げる

In [12]:
import requests

# API の Endpoint
url = 'http://127.0.0.1:5000/api'
# Request で渡す Data
files = {
    'students': open('resource/students.csv', 'r'),
    'cars': open('resource/cars.csv', 'r')
}
# POST request
response = requests.post(url, files=files)
# 結果の保存（`response.text` で Response 内容に access 可能）
with open('resource/solution_requests.csv', 'w') as f:
    f.write(response.text)